In [1]:
import boto3
import json
import pandas as pd
import re
from openai import OpenAI


def answer_question(question, prompt_data, tokens=1000):
    modelId = "gpt-4o"

    input = {
        "modelId": modelId,
        "contentType": "application/json",
        "accept": "*/*"
    }
    
    completion = client.chat.completions.create(
        model=input['modelId'],
        messages=[
            {
                "role": "user",
                "content": prompt_data + " " + question
            }
        ]
    )

    response_body = completion.choices[0].message.content

    return response_body


def get_sentiment(candidato, text, prompt):
    question = f"""
    Candidato: {candidato}
    Noticia: {text}
    """
    
    text = answer_question(question, prompt)
    
    data = {
        "thinking": re.search(r"<thinking>(.*?)</thinking>", text, re.DOTALL).group(1).strip(),
        "tono": re.search(r"<tono>(.*?)</tono>", text).group(1).strip(),
        "reason": re.search(r"<reason>(.*?)</reason>", text).group(1).strip(),
        "key_words": re.search(r"<key_words>(.*?)</key_words>", text).group(1).strip()
    }
    
    # Convertir a DataFrame
    return data["thinking"], data["tono"], data["reason"], data["key_words"]

client = OpenAI(api_key='xxxxx')


In [2]:
prompt = open('prompt.txt', 'r').read()

In [3]:
df = pd.read_csv("datos_politica.csv")
df = df.drop_duplicates()
df = df[~df['articleBody'].isna()]

candidato = "Sergio Fajardo"
df_fajardo = df[(df['articleBody'].str.lower().str.contains('fajardo')) & \
                (df['articleBody'].str.lower().str.contains('sergio'))].reset_index(drop=True)

df_fajardo[['thinking', 'tono', 'reason', 'keywords_llm']] = ['', '', '', '']


In [4]:

for i, noticia in enumerate(df_fajardo['articleBody']):
    thinking, tono, reason, keywords_llm = get_sentiment(candidato, noticia, prompt)
    df_fajardo.loc[i, 'thinking'] = thinking
    df_fajardo.loc[i, 'tono'] = tono
    df_fajardo.loc[i, 'reason'] = reason
    df_fajardo.loc[i, 'keywords_llm'] = keywords_llm
    print(f"Getting sentiment of article {i} out of {len(df_fajardo)}. Tono: {tono}")

    df_fajardo.to_csv("datos_fajardo.csv", index=False, encoding='utf-8-sig')


Getting sentiment of article 0 out of 144. Tono: Neutro
Getting sentiment of article 1 out of 144. Tono: Neutro
Getting sentiment of article 2 out of 144. Tono: Neutro
Getting sentiment of article 3 out of 144. Tono: Neutro
Getting sentiment of article 4 out of 144. Tono: Neutro
Getting sentiment of article 5 out of 144. Tono: Neutro
Getting sentiment of article 6 out of 144. Tono: Neutro
Getting sentiment of article 7 out of 144. Tono: Neutro
Getting sentiment of article 8 out of 144. Tono: Neutro
Getting sentiment of article 9 out of 144. Tono: Positivo
Getting sentiment of article 10 out of 144. Tono: Positivo
Getting sentiment of article 11 out of 144. Tono: Neutro
Getting sentiment of article 12 out of 144. Tono: Neutro
Getting sentiment of article 13 out of 144. Tono: Neutro
Getting sentiment of article 14 out of 144. Tono: Positivo
Getting sentiment of article 15 out of 144. Tono: Positivo
Getting sentiment of article 16 out of 144. Tono: Neutro
Getting sentiment of article 17 o

In [5]:
df_fajardo.value_counts('tono')

tono
Neutro      100
Negativo     27
Positivo     17
Name: count, dtype: int64